In [1]:
# 转置卷积
''' 
到⽬前为⽌，我们所⻅到的卷积神经⽹络层，例如卷积层（6.2节）和汇聚层（6.5节），通常会减少下采样输
⼊图像的空间维度（⾼和宽）。然⽽如果输⼊和输出图像的空间维度相同，在以像素级分类的语义分割中将
会很⽅便。例如，输出像素所处的通道维可以保有输⼊像素在同⼀位置上的分类结果

为了实现这⼀点，尤其是在空间维度被卷积神经⽹络层缩⼩后，我们可以使⽤另⼀种类型的卷积神经⽹络
层，它可以增加上采样中间层特征图的空间维度。在本节中，我们将介绍 转置卷积（transposed convolution）
[Dumoulin & Visin, 2016]，⽤于逆转下采样导致的空间尺⼨减⼩
'''
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# 基本操作
''' 
让我们暂时忽略通道，从基本的转置卷积开始，设步幅为1且没有填充。假设我们有⼀个nh × nw的输⼊张量
和⼀个kh × kw的卷积核。以步幅为1滑动卷积核窗⼝，每⾏nw次，每列nh次，共产⽣nhnw个中间结果。每个
中间结果都是⼀个(nh + kh − 1) × (nw + kw − 1)的张量，初始化为0。为了计算每个中间张量，输⼊张量中
的每个元素都要乘以卷积核，从⽽使所得的kh × kw张量替换中间张量的⼀部分。请注意，每个中间张量被替
换部分的位置与输⼊张量中元素的位置相对应。最后，所有中间结果相加以获得最终结果

我们可以对输⼊矩阵X和卷积核矩阵K实现基本的转置卷积运算trans_conv
'''
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1)) # 结果
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
    return Y

In [3]:
''' 
与通过卷积核“减少”输⼊元素的常规卷积（在 6.2节中）相⽐，转置卷积通过卷积核“⼴播”输⼊元素，从⽽
产⽣⼤于输⼊的输出。我们可以通过 图13.10.1来构建输⼊张量X和卷积核张量K从⽽验证上述实现输出。此
实现是基本的⼆维转置卷积运算
'''
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [4]:
'或者，当输⼊X和卷积核K都是四维张量时，我们可以使⽤⾼级API获得相同的结果'
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [5]:
# 填充步幅和多通道
''' 
与常规卷积不同，在转置卷积中，填充被应⽤于的输出（常规卷积将填充应⽤于输⼊）。例如，当将⾼和宽两
侧的填充数指定为1时，转置卷积的输出中将删除第⼀和最后的⾏与列
'''
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [6]:
'''
在转置卷积中，步幅被指定为中间结果（输出），⽽不是输⼊。使⽤ 图13.10.1中相同输⼊和卷积核张量，将
步幅从1更改为2会增加中间张量的⾼和权重，因此输出张量在 图13.10.2中
'''
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [7]:
''' 
对于多个输⼊和输出通道，转置卷积与常规卷积以相同⽅式运作。假设输⼊有ci个通道，且转置卷积为每个
输⼊通道分配了⼀个kh × kw的卷积核张量。当指定多个输出通道时，每个输出通道将有⼀个ci × kh × kw的
卷积核。
同样，如果我们将X代⼊卷积层f来输出Y = f(X)，并创建⼀个与f具有相同的超参数、但输出通道数量是X中
通道数的转置卷积层g，那么g(Y )的形状将与X相同。下⾯的⽰例可以解释这⼀点
'''
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3) # 常规卷积
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3) # 转置卷积
tconv(conv(X)).shape == X.shape # 常规卷积的转置卷积形状与X相同

True

In [8]:
# 与矩阵变换的关系
''' 
转置卷积为何以矩阵变换命名呢？让我们⾸先看看如何使⽤矩阵乘法来实现卷积。在下⾯的⽰例中，我们定
义了⼀个3 × 3的输⼊X和2 × 2卷积核K，然后使⽤corr2d函数计算卷积输出Y
'''
X = torch.arange(9.0).reshape(3, 3) # 3 × 3的输⼊
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]]) # 2 × 2卷积核
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [9]:
'''
接下来，我们将卷积核K重写为包含⼤量0的稀疏权重矩阵W。权重矩阵的形状是（4，9），其中⾮0元素来⾃
卷积核K
'''
def kernel2matrix(K):
    k, W = torch.zeros(5), torch.zeros((4, 9))
    k[:2], k[3:5] = K[0, :], K[1, :]
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    return W
W = kernel2matrix(K) # 权重矩阵
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [10]:
''' 
逐⾏连结输⼊X，获得了⼀个⻓度为9的⽮量。然后，W的矩阵乘法和向量化的X给出了⼀个⻓度为4的向量。重
塑它之后，可以获得与上⾯的原始卷积操作所得相同的结果Y：我们刚刚使⽤矩阵乘法实现了卷积
'''
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

In [11]:
''' 
同样，我们可以使⽤矩阵乘法来实现转置卷积。在下⾯的⽰例中，我们将上⾯的常规卷积2 × 2的输出Y作为
转置卷积的输⼊。想要通过矩阵相乘来实现它，我们只需要将权重矩阵W的形状转置为(9, 4)
'''
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [ ]:
''' 
抽象来看，给定输⼊向量x和权重矩阵W，卷积的前向传播函数可以通过将其输⼊与权重矩阵相乘并输出向
量y = Wx来实现。由于反向传播遵循链式法则和∇xy = W⊤，卷积的反向传播函数可以通过将其输⼊与转置
的权重矩阵W⊤相乘来实现。因此，转置卷积层能够交换卷积层的正向传播函数和反向传播函数：它的正向
传播和反向传播函数将输⼊向量分别与W⊤和W相乘
--------------------------------------------------------------------------------------
⼩结
• 与通过卷积核减少输⼊元素的常规卷积相反，转置卷积通过卷积核⼴播输⼊元素，从⽽产⽣形状⼤于
输⼊的输出
• 如果我们将X输⼊卷积层f来获得输出Y = f(X)并创造⼀个与f有相同的超参数、但输出通道数是X中通
道数的转置卷积层g，那么g(Y )的形状将与X相同
• 我们可以使⽤矩阵乘法来实现卷积。转置卷积层能够交换卷积层的正向传播函数和反向传播函数

'''